# Using SQL for Business Analysis
The Chinook Record Store signed a deal with a new record label, and the task is selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

|Artist Name |	Genre |
|-----|-----|
|Regal| Hip-Hop|
|Red Tone| Punk|
|Meteor and the Girls|Pop |
|Slim Jim Bites |Blues |

The Chinook database is provided as a SQLite database file called chinook.db. The database can be represented as the scheme below:

![img](https://s3.amazonaws.com/dq-content/191/chinook-schema.svg)

We'll use the following code to connect our Jupyter Notebook to our database file

## Connecting the Database

In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [3]:
%%sql
-- A query to return information on the tables and views in the database.
SELECT    name,  type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


 ## Selecting Albums to Purchase
 
 We need to write a query to find out which genres sell the most tracks. 

In [26]:
%%sql
SELECT g.name Genre, CAST(SUM(il.quantity) AS Integer) Quantity, 
ROUND(100.0 * SUM(il.quantity) / (SELECT COUNT(*) FROM invoice_line), 2) AS Percentage

FROM invoice_line il

LEFT JOIN track t ON t.track_id = il.track_id
LEFT JOIN genre g ON g.genre_id = t.genre_id

GROUP BY 1
ORDER BY 2 DESC;


Done.


Genre,Quantity,Percentage
Rock,2635,55.39
Metal,619,13.01
Alternative & Punk,492,10.34
Latin,167,3.51
R&B/Soul,159,3.34
Blues,124,2.61
Jazz,121,2.54
Alternative,117,2.46
Easy Listening,74,1.56
Pop,63,1.32


Based on sales of tracks from these genres : 
Rock with 55%, Metal with 13% and  Alternative & Punk (10.34), which account for approximately 78% of sales. Only on of the three artists whose albums we should purchase for the store fits into this niche, `Red Tone - Punk`. 

We need to investigate this sales further. 

Each customer of the Chinook store gets assigned to a sales support agent within the company when they first make a purchase.We need to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

## Analyzing Employee Sales Performance

In [5]:
%%sql
SELECT *
FROM employee;

Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
6,Mitchell,Michael,IT Manager,1,1973-07-01 00:00:00,2016-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com
7,King,Robert,IT Staff,6,1970-05-29 00:00:00,2017-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com
8,Callahan,Laura,IT Staff,6,1968-01-09 00:00:00,2017-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com


In [6]:
%%sql
SELECT e.first_name || ' ' || e.last_name AS 'Name' , 
ROUND(SUM(i.total), 2) AS 'Total Sales', hire_date 'Hire Date',
COUNT(e.employee_id) 'Total Customers'
FROM invoice i

LEFT JOIN customer c ON c.customer_id = i.customer_id
LEFT JOIN employee e ON e.employee_id = c.support_rep_id

GROUP BY e.employee_id
ORDER BY 2 DESC;

Done.


Name,Total Sales,Hire Date,Total Customers
Jane Peacock,1731.51,2017-04-01 00:00:00,212
Margaret Park,1584.0,2017-05-03 00:00:00,214
Steve Johnson,1393.92,2017-10-17 00:00:00,188


There are three Sales Support Agents and Jane Peacock has the highest total sales (1731.51) followed by Margaret Park (1584.0) then Steve Johnson (1393.92). Assuming that the total sales directly correlate to a particular agent, Jane Peacock has the most experience working at the company and this may be a facor to attribute to the high sales records considering that Steve Johnson is the least experienced.

## Analyzing Sales by Country

The next task is to analyze the sales data for customers from each different country. We will use the country value from the customers table, and ignore the country from the billing address in the invoice table. In particular, we will calculate data, for each country, on the:
- total number of customers
- total value of sales
- average value of sales per customer


In [100]:
%%sql
SELECT c.country 'Country' , ROUND(SUM(i.total), 2) AS 'Total Sales', 
COUNT(c.customer_id) AS 'Number of Customers', 
ROUND(SUM(i.total) / COUNT(c.customer_id),2) AS 'Average Value of Sales Per Customer'
FROM invoice i

JOIN customer c ON c.customer_id = i.customer_id

GROUP BY 1
ORDER BY 4 DESC;

Done.


Country,Total Sales,Number of Customers,Average Value of Sales Per Customer
Czech Republic,273.24,30,9.11
Spain,98.01,11,8.91
Ireland,114.84,13,8.83
United Kingdom,245.52,28,8.77
India,183.15,21,8.72
Belgium,60.39,7,8.63
Germany,334.62,41,8.16
Australia,81.18,10,8.12
Norway,72.27,9,8.03
USA,1040.49,131,7.94


When we rank countries by their average value of sales per customer, showing us that Czech consumers are by far the most willing to spend the most.

## Albums vs Individual Tracks

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

- purchase a whole album
- purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We need a query that categorizes each invoice as either an album purchase or not, and calculates the following summary statistics:
- Number of invoices
- Percentage of invoices

In order to answer the question, we're going to have to identify whether each invoice has all the tracks from an album. We can do this by getting the list of tracks from an invoice and comparing it to the list of tracks from an album. We can find the album to compare the purchase to by looking up the album that one of the purchased tracks belongs to. It doesn't matter which track we pick, since if it's an album purchase, that album will be the same for all tracks.

In [8]:
%%sql 
WITH
    album_track AS(
    SELECT a.*, t.track_id
    FROM album a
    INNER JOIN track t ON t.album_id = a.album_id),
    
    album_track_sum AS (
    SELECT at.album_id, SUM(track_id) + album_id sum_id
    FROM  album_track at
    GROUP BY 1 ),
 
    invoice_ln AS (
    SELECT *
    FROM invoice_line il
    INNER JOIN invoice i ON i.invoice_id=il.invoice_id
    INNER JOIN track t ON t.track_id = il.track_id
    ),

    sum_at AS (
    SELECT
        album_track.*,
        album_track_sum.sum_id
    FROM album_track
    INNER JOIN  album_track_sum ON  album_track_sum.album_id =  album_track.album_id),

    final AS (
    SELECT ilt.invoice_id, sat.album_id,  sat.artist_id,
        sat.sum_id,  sat.track_id, ilt.quantity
    FROM sum_at sat
    INNER JOIN invoice_ln ilt ON ilt.track_id = sat.track_id),

    combined AS (
    SELECT invoice_id,
        COUNT(DISTINCT(album_id)) 'Albums per Sale',
        COUNT(track_id) 'Tracks per Sale', sum_id,
        SUM(track_id)+album_id full_compare,
    CASE
        WHEN (SUM(track_id)+album_id) IS sum_id
        THEN 1
        ELSE 0
    END AS full_album_sold
    FROM final
    GROUP BY 1
        )
    
SELECT
    full_album_sold, 
    COUNT(invoice_id) 'Total Sales',
    ROUND((COUNT(invoice_id)*100.0)/(SELECT COUNT(sum_id) FROM combined), 2) '% Sales'
FROM combined
GROUP BY 1;

Done.


full_album_sold,Total Sales,% Sales
0,500,81.43
1,114,18.57


Full albums account for 18.5% of purchases, for this reason I'd recommend that customers should be able to purchase full albums from record companies.The Chinook store should continue to buy full albums from record companies

### Number of tracks that have been purchased vs not purchased.

In [89]:
%%sql
WITH
    purchases_il AS (
    SELECT il.track_id,
        SUM(il.quantity) quantity
    FROM invoice_line il
    GROUP BY 1 ),
    
    tracks AS ( 
    SELECT t.track_id, pil.quantity
    FROM track t
    LEFT JOIN purchases_il pil ON pil.track_id = t.track_id
    ORDER BY 2 )
    
SELECT 
    quantity 'Quantity',  CAST(SUM(quantity) AS Int) 'Sold Tracks', COUNT(track_id) 'Total tracks Purchased',
    ROUND(COUNT(track_id)*100/(SELECT COUNT(track_id) FROM tracks), 2) '% Tracks Purchased'
FROM tracks
GROUP BY 1 ORDER BY 1;

Done.


Quantity,Sold Tracks,Total tracks Purchased,% Tracks Purchased
None,None,1697,48.0
1,705,705,20.0
2,876,438,12.0
3,732,244,6.0
4,564,141,4.0
5,490,98,2.0
6,420,70,1.0
7,238,34,0.0
8,232,29,0.0
9,171,19,0.0


**We can see that 48% of the available tracks have never been purchased. This implies that almost half of the tracks are not generating revenue. **

In [102]:
%%sql
WITH
    purchases_il AS (
    SELECT il.track_id,
        SUM(il.quantity) quantity
    FROM invoice_line il
    GROUP BY 1 ),
    
    tracks AS ( 
    SELECT t.track_id, pil.quantity, g.*
    FROM track t
    LEFT JOIN purchases_il pil ON pil.track_id = t.track_id
    LEFT JOIN genre g ON g.genre_id =t.genre_id)
    
SELECT 
    name 'Genre',
    CASE  WHEN SUM(quantity) IS NULL THEN 0
    ELSE SUM(quantity)
    END AS 'Tracks Sold', COUNT(track_id) 'Total tracks Purchased',
    ROUND(COUNT(track_id)*100.0/(SELECT COUNT(track_id) FROM tracks), 2) '% Tracks Available',    
    ROUND((SUM(quantity)*100.0/(SELECT SUM(quantity) FROM tracks)), 2) '% Tracks Sales'
FROM tracks
GROUP BY 1 
ORDER BY 2 DESC ;

Done.


Genre,Tracks Sold,Total tracks Purchased,% Tracks Available,% Tracks Sales
Rock,2635,1297,37.03,55.39
Metal,619,374,10.68,13.01
Alternative & Punk,492,332,9.48,10.34
Latin,167,579,16.53,3.51
R&B/Soul,159,61,1.74,3.34
Blues,124,81,2.31,2.61
Jazz,121,130,3.71,2.54
Alternative,117,40,1.14,2.46
Easy Listening,74,24,0.69,1.56
Pop,63,48,1.37,1.32


There are cases where full genres have never been sold: 'World', 'SciFI & Fantasy', 'Comedy', 'Bossa Nova', 'Science Fiction', 'Rock and Roll', 'Opera', which account for 3.2% of the available content.

We can see Drama, Soundtrack, Heavy Metal, Latin, and TV Shows have low sales compaired to the available tracks purchased.

On the other hand, when ranking genres by their representation in sales, Easy Listening, Alternative, R&B/Soul and Metal ought to double their purchases given their higher sales records

## Conclusion
The aim of this project was answering business questions using SQL on the Chinook database. We can make the following conclusions from our observations:

- The most popular music genre is by far Rock (2635), accounting for 55% of the sales and revenue; Metal (619) and Alternative & Punk (492).
- We can attribute the difference in sales assigned to each agent to experience.
- Czech consumers are by far the most willing to spend.
- Approximately 18.5% of the sales are full albums.
- There are cases where full genres have never been sold: 'World', 'SciFI & Fantasy', 'Comedy', 'Bossa Nova', 'Science Fiction', 'Rock and Roll', 'Opera', which account for 3.2% of the available content.
- To generate more revenue these genres need more represention (Metal, Blues, Easy Listening, R&B/Soul, Alternative), these are over purchased (Drama, TV Shows, Soundtracks).

Based on these findings I could recommend that albums from these three artists should be added to the store based on their genre:
1. Red Tone (Punk)
2. Slim Jim Bites	(Blues) 
3. Meteor and the Girls(Pop)